# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Нам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Построим модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверим *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измерим *AUC-ROC*, сравнивая её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# Описание данных

Признаки

- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — сколько лет человек является клиентом банка
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

Целевой признак

- Exited — факт ухода клиента

## Подготовка данных

In [1]:
#Импортируем необходимые библиотеки
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import warnings
warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None

In [2]:
#Откроем файл с данными
df = pd.read_csv('/datasets/Churn.csv')

In [3]:
#Посмотрим первые 10 строк
df.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [4]:
#Изучим общую информацию
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [5]:
#Приведём заголовки к нижнему регистру и слегка переименуем их
df = df.rename(columns = {'RowNumber': 'row_number', 'CustomerId': 'customer_id', 'Surname': 'surname', 
                          'CreditScore': 'credit_score', 'Geography': 'geography', 'Gender': 'gender', 'Age' : 'age', 
                          'Tenure' : 'tenure', 'Balance': 'balance', 'NumOfProducts': 'num_of_products', 
                          'HasCrCard': 'has_cr_card', 'IsActiveMember': 'is_active_member', 
                          'EstimatedSalary': 'estimated_salary', 'Exited': 'exited'})

In [6]:
#Проверим изменения
df.columns

Index(['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
       'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card',
       'is_active_member', 'estimated_salary', 'exited'],
      dtype='object')

In [7]:
#Посмотрим, сколько пропусков имеет столбец 'tenure'
df['tenure'].isna().sum()

909

In [8]:
#Посмотрим, какие значения в столбце 'tenure'
df['tenure'].value_counts()

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: tenure, dtype: int64

In [9]:
#Заменим тип данных столбца 'tenure' на 'Int64'
df['tenure'] = df['tenure'].astype('Int64')

In [10]:
#Удалим данные с пропуском
df = df.dropna(subset = ['tenure'], axis=0)

In [11]:
#Посмотрим количество дубликатов
df.duplicated().sum()

0

In [12]:
#Удалим ненужные данные для алгоритма
for_drop = ['row_number','customer_id', 'surname']
df_with_drop = df.drop(for_drop, axis=1)
df_with_drop.head()

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [13]:
#Посмотрим уникальные значения столбца 'geography'
df['geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [14]:
#Посмотрим уникальные значения столбца 'gender'
df['gender'].unique()

array(['Female', 'Male'], dtype=object)

In [15]:
#Применим метод прямого кодирования и избежим дамми-ловушек
df_with_drop = pd.get_dummies(df_with_drop, drop_first=True)
df_with_drop.head(10)

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0
5,645,44,8,113755.78,2,1,0,149756.71,1,0,1,1
6,822,50,7,0.00,2,1,1,10062.80,0,0,0,1
7,376,29,4,115046.74,4,1,0,119346.88,1,1,0,0
8,501,44,4,142051.07,2,0,1,74940.50,0,0,0,1
9,684,27,2,134603.88,1,1,1,71725.73,0,0,0,1


### Вывод

Были исправление заголовки столбцов, изменён тип данных для столбца 'tenure', удалены пропуски, дубликаты отсутствуют. 

Также был применён метод прямого кодирования с избеганием дамми-ловушек, удалены ненужные столбцы.

## Исследование задачи

In [16]:
#Разделим наши признаки и выделим целевой
features = df_with_drop.drop('exited', axis=1)
target = df_with_drop['exited']

In [17]:
#Разобьём наши данные на тренировочную, валидационную и тестовую выборки
features_train, features_val, target_train, target_val = train_test_split(features, target, test_size=0.40, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_val, target_val, 
                                                                            test_size=0.50, random_state=12345) 

In [18]:
#Посмотрим размер каждой выборки
print('Размер тренировочной выборки:', features_train.shape, target_train.shape)
print('Размер валидационной выборки:', features_valid.shape, target_valid.shape)
print('Размер тестовой выборки:', features_test.shape, target_test.shape)

Размер тренировочной выборки: (5454, 11) (5454,)
Размер валидационной выборки: (1818, 11) (1818,)
Размер тестовой выборки: (1819, 11) (1819,)


In [19]:
#Масштамбируем признаки для каждой выборки, выделим необходимые столбцы
numeric = ['credit_score', 'age', 'tenure', 'balance', 'num_of_products', 'estimated_salary']

In [20]:
#Масштамбируем для тренировочной выборки
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_train.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
9344,0.809075,-1.039327,-1.025995,0.554904,-0.908179,1,0,0.019508,0,0,0
3796,-1.152518,-1.227561,0.696524,0.480609,-0.908179,0,0,0.056167,0,0,1
7462,-0.398853,0.090079,1.385532,-1.237830,-0.908179,1,1,0.848738,0,0,1
1508,-0.749875,-0.286389,0.352020,-1.237830,0.809300,1,1,-0.894953,0,0,1
4478,-1.028628,-0.756975,-0.336987,-1.237830,0.809300,0,1,-1.284516,0,0,1


In [21]:
#Масштамбируем для валидационной выборки
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_valid.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
7445,-1.369326,0.560665,-0.336987,-1.237830,-0.908179,1,0,-0.086537,0,0,0
8620,1.232367,0.090079,1.041028,-1.237830,0.809300,0,1,-0.537457,0,0,0
1714,0.840048,0.560665,0.352020,1.231363,-0.908179,0,0,1.070393,1,0,1
5441,1.056856,-0.945210,-1.370498,0.951231,-0.908179,1,0,-0.576279,0,0,1
9001,0.406433,-0.662858,0.352020,0.767800,-0.908179,1,1,0.662068,0,1,1


In [22]:
#Масштамбируем для тестовой выборки
features_test[numeric] = scaler.transform(features_test[numeric])
features_test.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,geography_Germany,geography_Spain,gender_Male
5170,1.707279,-0.756975,-0.336987,-1.237830,0.809300,1,1,0.718362,0,0,0
4180,-0.429826,-0.286389,1.730036,0.485538,0.809300,0,0,1.687305,1,0,1
7349,-0.171721,0.278313,0.352020,-0.269213,0.809300,0,1,0.824128,1,0,1
7469,0.385784,-0.380507,1.041028,0.464813,-0.908179,1,0,-1.118018,0,0,0
3467,-1.142194,0.278313,-1.370498,0.353837,0.809300,0,1,-0.107642,1,0,0


In [23]:
#Рассмотрим модель решающего дерева
model_DTC = DecisionTreeClassifier(random_state=12345, max_depth=100)
model_DTC.fit(features_train, target_train)
predictions_valid = model_DTC.predict(features_valid)
accuracy_DTC = accuracy_score(target_valid, predictions_valid)
f1_score_DTC = f1_score(target_valid, predictions_valid)
auc_roc_DTC = roc_auc_score(target_valid, model_DTC.predict_proba(features_valid)[:, 1])
print("Точность:", accuracy_DTC)
print('F1-мера:', f1_score_DTC)
print('AUC-ROC:', auc_roc_DTC)

Точность: 0.7904290429042904
F1-мера: 0.4926764314247669
AUC-ROC: 0.6797786314764468


In [24]:
#Рассмотрим модель случайного леса
model_RFC = RandomForestClassifier(random_state=12345, n_estimators=100)
model_RFC.fit(features_train, target_train)
predictions_valid = model_RFC.predict(features_valid)
accuracy_RFC = accuracy_score(target_valid, predictions_valid)
f1_score_RFC = f1_score(target_valid, predictions_valid)
auc_roc_RFC = roc_auc_score(target_valid, model_RFC.predict_proba(features_valid)[:, 1])
print("Точность:", accuracy_RFC)
print('F1-мера:', f1_score_RFC)
print('AUC-ROC:', auc_roc_RFC)

Точность: 0.8619361936193619
F1-мера: 0.5851239669421487
AUC-ROC: 0.8629313566139046


In [25]:
#Рассмотрим модель логистической регрессии
model_LR = LogisticRegression(solver="liblinear", random_state=12345)
model_LR.fit(features_train, target_train)
predictions_valid = model_LR.predict(features_valid)
accuracy_LR = accuracy_score(target_valid, predictions_valid)
f1_score_LR = f1_score(target_valid, predictions_valid)
auc_roc_LR = roc_auc_score(target_valid, model_LR.predict_proba(features_valid)[:, 1])
print("Точность:", accuracy_LR)
print('F1-мера:', f1_score_LR)
print('AUC-ROC:', auc_roc_LR)

Точность: 0.8085808580858086
F1-мера: 0.30400000000000005
AUC-ROC: 0.7736191158144302


In [26]:
#Проверим баланс классов для тренировочной выборки
target_train.value_counts(normalize=1)

0    0.793546
1    0.206454
Name: exited, dtype: float64

In [27]:
#Проверим баланс классов для валидационной выборки
target_valid.value_counts(normalize=1)

0    0.792629
1    0.207371
Name: exited, dtype: float64

### Вывод

Данные разбили следующим образом:

- 60% данных - train;
- 20% данных - valid;
- 20% данных - test;

Обучение будет происходить на данных из train.

Валидация будет происходить на данных из valid.

Лучшая модель по валидации будет применена на данных test.

Баланс классов составляет примерно 80% для значения "0" и 20% для значения "1", надо исправить этот дисбаланс.

## Борьба с дисбалансом

In [28]:
#Для борьбы с дисбалансом воспользуемся методом увеличения выборки, для этого напишем функцию
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [29]:
#Применим нашу функцию для модели случайного леса
features_upsampled_train, target_upsampled_train = upsample(features_train, target_train, repeat=4)

model_RFC = RandomForestClassifier(n_estimators=100, random_state=12345)
model_RFC.fit(features_upsampled_train, target_upsampled_train)
predicted_valid = model_RFC.predict(features_valid)
print('Accuracy:', accuracy_score(target_valid, predicted_valid))
print('Precision:', precision_score(target_valid, predicted_valid))
print('Recall:', recall_score(target_valid, predicted_valid))
print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, model_RFC.predict_proba(features_valid)[:, 1]))

Accuracy: 0.8536853685368537
Precision: 0.6831683168316832
Recall: 0.5490716180371353
F1: 0.6088235294117647
AUC-ROC: 0.8590492161168656


In [30]:
#Применим нашу функцию для модели решающего дерева
model_DTC = DecisionTreeClassifier(random_state=12345, max_depth=100)
model_DTC.fit(features_upsampled_train, target_upsampled_train)
predicted_valid = model_DTC.predict(features_valid)
print('Accuracy:', accuracy_DTC)
print('Precision:', precision_score(target_valid, predicted_valid))
print('Recall:', recall_score(target_valid, predicted_valid))
print('F1-мера:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, model_DTC.predict_proba(features_valid)[:, 1]))

Accuracy: 0.7904290429042904
Precision: 0.4735376044568245
Recall: 0.4509283819628647
F1-мера: 0.46195652173913043
AUC-ROC: 0.6598847322722027


In [31]:
#Применим нашу функцию для модели логистической регрессии
model_LR = LogisticRegression(solver="liblinear", random_state=12345)
model_LR.fit(features_upsampled_train, target_upsampled_train)
predicted_valid = model_LR.predict(features_valid)
print('Accuracy:', accuracy_LR)
print('Precision:', precision_score(target_valid, predicted_valid))
print('Recall:', recall_score(target_valid, predicted_valid))
print('F1-мера:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, model_LR.predict_proba(features_valid)[:, 1]))

Accuracy: 0.8085808580858086
Precision: 0.385989010989011
Recall: 0.7453580901856764
F1-мера: 0.5085972850678734
AUC-ROC: 0.7779006989325494


In [32]:
#Воспользуемся методом уменьшения выборки, для этого напишем функцию
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [33]:
#Применим нашу функцию для модели случайного леса
features_downsampled_train, target_downsampled_train = downsample(features_train, target_train, fraction=0.25)

model_RFC = RandomForestClassifier(n_estimators=100, random_state=12345)
model_RFC.fit(features_downsampled_train, target_downsampled_train)
predicted_valid = model_RFC.predict(features_valid)
print('Accuracy:', accuracy_score(target_valid, predicted_valid))
print('Precision:', precision_score(target_valid, predicted_valid))
print('Recall:', recall_score(target_valid, predicted_valid))
print('F1:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, model_RFC.predict_proba(features_valid)[:, 1]))

Accuracy: 0.7678767876787679
Precision: 0.4652241112828439
Recall: 0.7984084880636605
F1: 0.5878906250000001
AUC-ROC: 0.8639851856487814


In [34]:
#Применим нашу функцию для модели решающего дерева
model_DTC = DecisionTreeClassifier(random_state=12345, max_depth=100)
model_DTC.fit(features_downsampled_train, target_downsampled_train)
predicted_valid = model_DTC.predict(features_valid)
print('Accuracy:', accuracy_DTC)
print('Precision:', precision_score(target_valid, predicted_valid))
print('Recall:', recall_score(target_valid, predicted_valid))
print('F1-мера:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, model_DTC.predict_proba(features_valid)[:, 1]))

Accuracy: 0.7904290429042904
Precision: 0.38419618528610355
Recall: 0.7480106100795756
F1-мера: 0.5076507650765076
AUC-ROC: 0.7171697741584554


In [35]:
#Применим нашу функцию для модели логистической регрессии
model_LR = LogisticRegression(solver="liblinear", random_state=12345)
model_LR.fit(features_downsampled_train, target_downsampled_train)
predicted_valid = model_LR.predict(features_valid)
print('Accuracy:', accuracy_LR)
print('Precision:', precision_score(target_valid, predicted_valid))
print('Recall:', recall_score(target_valid, predicted_valid))
print('F1-мера:', f1_score(target_valid, predicted_valid))
print('AUC-ROC:', roc_auc_score(target_valid, model_LR.predict_proba(features_valid)[:, 1]))

Accuracy: 0.8085808580858086
Precision: 0.3812754409769335
Recall: 0.7453580901856764
F1-мера: 0.5044883303411131
AUC-ROC: 0.7770355467117773


### Вывод

Показатели при увеличении выборки для модели решающего дерева:
- Точность: 0.7904290429042904
- Precision: 0.4735376044568245
- Recall: 0.4509283819628647
- F1-мера: 0.46195652173913043
- AUC-ROC: 0.6598847322722027

Показатели при увеличении выборки для модели случайного леса:
- Accuracy: 0.8536853685368537
- Precision: 0.6831683168316832
- Recall: 0.5490716180371353
- F1: 0.6088235294117647
- AUC-ROC: 0.8590492161168656

Показатели при увеличении выборки для модели логистической регрессии:
- Accuracy: 0.8085808580858086
- Precision: 0.385989010989011
- Recall: 0.7453580901856764
- F1-мера: 0.5085972850678734
- AUC-ROC: 0.7779006989325494

Показатели при уменьшении выборки для модели решающего дерева:
- Accuracy: 0.7904290429042904
- Precision: 0.38419618528610355
- Recall: 0.7480106100795756
- F1-мера: 0.5076507650765076
- AUC-ROC: 0.7171697741584554

Показатели при уменьшении выборки для модели случайного леса:
- Accuracy: 0.7678767876787679
- Precision: 0.4652241112828439
- Recall: 0.7984084880636605
- F1: 0.5878906250000001
- AUC-ROC: 0.8639851856487814

Показатели при уменьшении выборки для модели логистической регрессии:
- Accuracy: 0.8085808580858086
- Precision: 0.3812754409769335
- Recall: 0.7453580901856764
- F1-мера: 0.5044883303411131
- AUC-ROC: 0.7770355467117773

Вывод: самая оптимальная модель по параметрам F1-меры и AUC-ROC - модель случайного леса с увеличением выборки.

## Тестирование модели

In [36]:
#Протестируем нашу модель случайного леса
model_RFC = RandomForestClassifier(n_estimators=100, random_state=12345)
model_RFC.fit(features_upsampled_train, target_upsampled_train)
predicted_test = model_RFC.predict(features_test)
print('Accuracy:', accuracy_score(target_test, predicted_test))
print('Precision:', precision_score(target_test, predicted_test))
print('Recall:', recall_score(target_test, predicted_test))
print('F1:', f1_score(target_test, predicted_test))
print('AUC-ROC:', roc_auc_score(target_test, model_RFC.predict_proba(features_test)[:, 1]))

Accuracy: 0.8614623419461243
Precision: 0.6655518394648829
Recall: 0.5669515669515669
F1: 0.6123076923076923
AUC-ROC: 0.8498101958592422


### Вывод

Были получены относительно высокие показатели, F1-мера > 0.59, AUC-ROC ~ 0.85.

## Общий вывод

Самая эффективная модель - модель случайного леса


Значения для модели случайного леса при дисбалансе классов:
- F1-мера: 0.5851239669421487
- AUC-ROC: 0.8629313566139046

Значения для модели случайного леса при увеличении выборки:
- F1: 0.6088235294117647
- AUC-ROC: 0.8590492161168656

Значения для модели случайного леса при уменьшении выборки:
- F1: 0.5878906250000001
- AUC-ROC: 0.8639851856487814

Значения на тестовой модели:
- F1: 0.6123076923076923
- AUC-ROC: 0.8498101958592422

Значение F1-меры > 0.59, AUC-ROC метрика была исследована на всех этапах задачи.